In [ ]:
import json
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from geopy import distance
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
driver_df = pd.read_csv('/content/drive/Shareddrives/Brillante Workspace/ML Corner/Dataset/kotlite_dataset/kotlite_driver_dataset_KWB_with_ket.csv')
driver_df.loc[1, 'pickuptime'] = '19-05-2021 06:30'
driver_df.loc[0, 'driver_id'] = 51
driver_df['pickuptime'] = pd.to_datetime(driver_df['pickuptime'])
driver_df.describe(include='all')

,driver_id,pickuptime,start_lat,start_long,end_lat,end_long,routes,ket_start,ket_end
count,20.000000,20,20.000000,20.000000,20.000000,20.000000,20,20,20
unique,NaN,13,NaN,NaN,NaN,NaN,20,19,17
top,NaN,2021-05-19 07:00:00,NaN,NaN,NaN,NaN,"[[-7.8927323, 112.5767376], [-7.8926213, 112.5...",SMA N 1 Batu,among tani
freq,NaN,4,NaN,NaN,NaN,NaN,1,2,3
first,NaN,2021-05-19 06:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2021-05-19 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,25.650000,NaN,-7.881579,112.539215,-7.888490,112.540150,NaN,NaN,NaN
std,16.239247,NaN,0.020704,0.023684,0.023849,0.029264,NaN,NaN,NaN
min,6.000000,NaN,-7.916181,112.508470,-7.952293,112.509902,NaN,NaN,NaN
25%,10.750000,NaN,-7.895160,112.526740,-7.905046,112.518075,NaN,NaN,NaN


In [ ]:
# create l1 distance function
def l1_distance(point1, point2):
  l1 = 0
  for i in range(len(point1)):
    l1 += abs(point1[i] - point2[i])
  return l1

# to get minimum point from start or end passanger point
def driver_data_handler(single_driv_data, passanger_point):
  # passanger point == [latitude, longitude]
  routes = single_driv_data[6]

  driver_routes = []

  # pop start point data to driver routes
  driver_routes.append([single_driv_data[2], single_driv_data[3]])
  
  # parse routes and iterate to pop driver point to driver routes
  route_parse = json.loads(routes)
  for _route in route_parse:
    driver_routes.append(_route)
  
  # pop end point data to driver routes
  driver_routes.append([single_driv_data[4], single_driv_data[5]])
  
  # after get driver_routes list, calculate distance
  dist = []
  for driver_point in driver_routes:
    l1_dist = l1_distance(passanger_point, driver_point)
    dist.append(l1_dist)
  
  # get minimum values info
  min_val = min(dist)
  idx = dist.index(min_val)
  nearest_point = driver_routes[idx]
  return min_val, idx, nearest_point

In [ ]:
# get nearest driver from passanger point
# query include [pickup_datetime, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude]
def nearest1st_step(driver_data, query):
  # nearest 1st step is use for get nearest distance, and in one way
  picktime = query[0]
  
  # create filter for 30 minutes before and after picktime
  diff = datetime.timedelta(hours=1)
  upth = picktime + diff
  lowth = picktime - diff
  
  # filter driver_data
  driver_data = driver_data.loc[(driver_data['pickuptime'] <= upth) & (driver_data['pickuptime'] >= lowth)]

  # get distance from routes point data
  dist = []
  for drdt in driver_data.values:
    start, _snear, pu_nearest_point = driver_data_handler(drdt, [query[1], query[2]])
    end, _enear, do_nearest_point = driver_data_handler(drdt, [query[3], query[4]])
    if _snear <= _enear:
      dist.append((start + end, drdt[0], pu_nearest_point, do_nearest_point))
  
  # sorting distance
  sorted_dist = sorted(dist)

  # list for recomendations
  recommendations = []
  # get real distance from pickup/dropoff point from snear or enear point
  for data in sorted_dist:
    # distance from pickup point to pu_nearest_point
    pick_distance = distance.distance([query[1], query[2]], data[2]).km
    drop_distance = distance.distance([query[3], query[4]], data[3]).km
    if (pick_distance <= 2.0) & (drop_distance <= 2.0):
      recommendations.append((data[1]))

  return recommendations

In [ ]:
driver_df[['ket_start', 'ket_end']]

,ket_start,ket_end
0,pendem,among tani
1,giripurno,SMA N 1 Batu
2,sisir,pascasarjana UIN
3,pokopek,UB
4,Dau,Museum Angkut
5,sumberejo,wonderland
6,Punten,among tani
7,Bumiaji,UMM
8,Pesanggrahan,Balijestro
9,"Wukir, temas",Krematorium


In [ ]:
pickup_time = '2021-05-19 07:15:00'
pickup_time = pd.to_datetime(pickup_time)
pass_query = [pickup_time,-7.8901831,112.5459769, -7.877689,112.52537]

In [ ]:
nearest1st_step(driver_df, pass_query)

[11]

In [ ]:
json.loads(driver_df.loc[0,'routes'])

[[-7.8927323, 112.5767376],
 [-7.8926213, 112.577466],
 [-7.892488600000001, 112.577469],
 [-7.8924153, 112.5786948],
 [-7.875366199999998, 112.5758507],
 [-7.869664699999999, 112.5659915],
 [-7.864168100000001, 112.5515171],
 [-7.862953599999999, 112.5268784],
 [-7.8677569, 112.524994],
 [-7.867638899999998, 112.5280067],
 [-7.870739899999999, 112.5275277],
 [-7.870086499999998, 112.5240687],
 [-7.8697798, 112.5230724],
 [-7.866768199999999, 112.5115954],
 [-7.867053099999999, 112.5111369]]

In [ ]:
driver_df.loc[0,:]

driver_id                                                    51
pickuptime                                  2021-05-19 07:00:00
start_lat                                              -7.89319
start_long                                              112.577
end_lat                                                -7.86698
end_long                                                112.511
routes        [[-7.8927323, 112.5767376], [-7.8926213, 112.5...
ket_start                                                pendem
ket_end                                              among tani
Name: 0, dtype: object

In [ ]:
driver_df[driver_df['driver_id'] == 11]

,driver_id,pickuptime,start_lat,start_long,end_lat,end_long,routes,ket_start,ket_end
4,11,2021-05-19 07:45:00,-7.916181,112.583958,-7.882215,112.511487,"[[-7.9161186, 112.5838508], [-7.91579580000000...",Dau,Museum Angkut
